In [1]:
import re
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow import keras
import requests
import urllib
from requests_html import HTML
from requests_html import HTMLSession
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import spacy

In [2]:
awards_model = keras.models.load_model('C:/Users/taige/Desktop/Research/summer2022/week8/django/mysite/classifier/resource_data/amodel_7.h5', custom_objects={'KerasLayer':hub.KerasLayer})
edu_model =  keras.models.load_model('C:/Users/taige/Desktop/Research/summer2022/week8/django/mysite/classifier/resource_data/emodel_6.h5', custom_objects={'KerasLayer':hub.KerasLayer})
interest_model = keras.models.load_model('C:/Users/taige/Desktop/Research/summer2022/week8/django/mysite/classifier/resource_data/imodel_5.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [54]:
cs = pd.read_csv('cs.csv', encoding = "ISO-8859-1", engine='python')
cs

,Int,Awd,Edu,Text
0,0,0,1,\nDivesh Aggarwal received his Ph.D. in Comput...
1,0,0,1,"From 2012 to 2016, he spent two years each as ..."
2,0,0,0,"Since 2016, he has been a faculty member in th..."
3,1,0,0,His research in the last decade has revolved a...
4,0,0,0,\n\n1.
...,...,...,...,...
135,0,0,1,He then began his doctoral work at the Univers...
136,0,0,0,(Erdos number = 3)\n\n
137,1,0,0,Ho-lun's interests include computational geome...
138,0,0,0,He was a teaching assistant during his master ...


In [55]:
escore = []
ascore = []
iscore = []
ep = []
ap = []
ip = []

for index, row in cs.iterrows():
    txt = row['Text']
    esc = edu_model.predict([txt])[0][0]
    escore.append(esc)

    if esc > 0.5:
        ep.append(1)
    else:
        ep.append(0)
    
    asc = awards_model.predict([txt])[0][0]
    ascore.append(asc)
    if asc > 0.5:
        ap.append(1)
    else:
        ap.append(0)
    
    isc = interest_model.predict([txt])[0][0]
    iscore.append(isc)

    if isc > 0.5:
        ip.append(1)
    else:
        ip.append(0)
        
cs['Edu Score'] = escore
cs['Awd Score'] = ascore
cs['Int Score'] = iscore

cs['Edu Prediction'] = ep
cs['Int Prediction'] = ip
cs['Awd Prediction'] = ap

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 35ms/step


In [56]:
cs

,Int,Awd,Edu,Text,Edu Score,Awd Score,Int Score,Edu Prediction,Int Prediction,Awd Prediction
0,0,0,1,\nDivesh Aggarwal received his Ph.D. in Comput...,0.801282,0.014967,0.003713,1,0,0
1,0,0,1,"From 2012 to 2016, he spent two years each as ...",0.390195,0.058694,0.152705,0,0,0
2,0,0,0,"Since 2016, he has been a faculty member in th...",0.236732,0.263940,0.199467,0,0,0
3,1,0,0,His research in the last decade has revolved a...,0.011356,0.008833,0.782592,0,1,0
4,0,0,0,\n\n1.,0.013388,0.063592,0.000646,0,0,0
...,...,...,...,...,...,...,...,...,...,...
135,0,0,1,He then began his doctoral work at the Univers...,0.923217,0.238028,0.085004,1,0,0
136,0,0,0,(Erdos number = 3)\n\n,0.058116,0.010944,0.006109,0,0,0
137,1,0,0,Ho-lun's interests include computational geome...,0.412545,0.051665,0.949655,0,1,0
138,0,0,0,He was a teaching assistant during his master ...,0.909886,0.269776,0.064877,1,0,0


In [57]:
cs[(cs['Edu Prediction'] == 1) & (cs['Edu'] == 0)]

,Int,Awd,Edu,Text,Edu Score,Awd Score,Int Score,Edu Prediction,Int Prediction,Awd Prediction
21,1,0,0,His doctorate research focused on human gait m...,0.758983,0.089493,0.425345,1,0,0
23,1,0,0,Prior to joining NUS he was employed at USP (T...,0.579739,0.037176,0.031454,1,0,0
30,0,0,0,"Subsequently, he was a postdoctoral associate ...",0.818653,0.305641,0.201907,1,0,0
50,0,0,0,"From 1996 to 1998, he was Research Associate a...",0.677959,0.352916,0.043545,1,0,0
65,0,0,0,\nTrevor E. Carlson is an Assistant Professor ...,0.630805,0.003229,0.103393,1,0,0
80,0,0,0,"After that, he spent two years at Charles Univ...",0.912870,0.282513,0.025089,1,0,0
82,0,0,0,\nDr. Chan Chee Yong is an Associate Professor...,0.527565,0.024149,0.101435,1,0,0
94,1,0,0,"In Bell Labs, he worked on network management ...",0.517249,0.040180,0.132464,1,0,0
134,0,0,0,in computer science at the Hong Kong Universit...,0.908681,0.210614,0.053113,1,0,0
138,0,0,0,He was a teaching assistant during his master ...,0.909886,0.269776,0.064877,1,0,0


In [58]:
def edu_performance(df):
    p = len(df[df['Edu'] == 1])
    n = len(df[df['Edu'] == 0])
    tp = len(df[(df['Edu Prediction'] == 1) & (df['Edu'] == 1)])
    tn = len(df[(df['Edu Prediction'] == 0) & (df['Edu'] == 0)])
    fp = len(df[(df['Edu Prediction'] == 1) & (df['Edu'] == 0)])
    fn = len(df[(df['Edu Prediction'] == 0) & (df['Edu'] == 1)])
    total = len(df)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print('---------------------')
    print('Edu performance')
    print('Sensitivity is: ', tp/p)
    print('Specificity is: ', tn/n)
    print('Accuracy is: ', (tp+tn)/total)
    print('Error rate is: ', (fp+fn)/total)
    print('Precision is: ', precision)
    print('Recall is: ', recall)
    print('F1 score is: ', (2*precision*recall)/(precision + recall))
    print('---------------------')
    
def int_performance(df):
    p = len(df[df['Int'] == 1])
    n = len(df[df['Int'] == 0])
    tp = len(df[(df['Int Prediction'] == 1) & (df['Int'] == 1)])
    tn = len(df[(df['Int Prediction'] == 0) & (df['Int'] == 0)])
    fp = len(df[(df['Int Prediction'] == 1) & (df['Int'] == 0)])
    fn = len(df[(df['Int Prediction'] == 0) & (df['Int'] == 1)])
    total = len(df)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print('---------------------')
    print('Int performance')
    print('Sensitivity is: ', tp/p)
    print('Specificity is: ', tn/n)
    print('Accuracy is: ', (tp+tn)/total)
    print('Error rate is: ', (fp+fn)/total)
    print('Precision is: ', precision)
    print('Recall is: ', recall)
    print('F1 score is: ', (2*precision*recall)/(precision + recall))
    print('---------------------')
    
def awd_performance(df):
    p = len(df[df['Awd'] == 1])
    n = len(df[df['Awd'] == 0])
    tp = len(df[(df['Awd Prediction'] == 1) & (df['Awd'] == 1)])
    tn = len(df[(df['Awd Prediction'] == 0) & (df['Awd'] == 0)])
    fp = len(df[(df['Awd Prediction'] == 1) & (df['Awd'] == 0)])
    fn = len(df[(df['Awd Prediction'] == 0) & (df['Awd'] == 1)])
    total = len(df)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print('---------------------')
    print('Awd performance')
    print('Sensitivity is: ', tp/p)
    print('Specificity is: ', tn/n)
    print('Accuracy is: ', (tp+tn)/total)
    print('Error rate is: ', (fp+fn)/total)
    print('Precision is: ', precision)
    print('Recall is: ', recall)
    print('F1 score is: ', (2*precision*recall)/(precision + recall))
    print('---------------------')
    
    

In [59]:
def print_edu_fp(df):
    fp = df[(df['Edu Prediction'] == 1) & (df['Edu'] == 0)]
    print('The average prediction score for false positive is: ', fp['Edu Score'].mean())
    for index, row in fp.iterrows():
        print('---------------------')
        print(row['Text'])
        
def print_int_fp(df):
    fp = df[(df['Int Prediction'] == 1) & (df['Int'] == 0)]
    print('The average prediction score for false positive is: ', fp['Int Score'].mean())
    for index, row in fp.iterrows():
        print('---------------------')
        print(row['Text'])
        
def print_awd_fp(df):
    fp = df[(df['Awd Prediction'] == 1) & (df['Awd'] == 0)]
    print('The average prediction score for false positive is: ', fp['Awd Score'].mean())
    for index, row in fp.iterrows():
        print('---------------------')
        print(row['Text'])

In [60]:
edu_performance(cs)
awd_performance(cs)
int_performance(cs)

---------------------
Edu performance
Sensitivity is:  0.8095238095238095
Specificity is:  0.9159663865546218
Accuracy is:  0.9
Error rate is:  0.1
Precision is:  0.6296296296296297
Recall is:  0.8095238095238095
F1 score is:  0.7083333333333334
---------------------
---------------------
Awd performance
Sensitivity is:  0.6470588235294118
Specificity is:  0.9349593495934959
Accuracy is:  0.9
Error rate is:  0.1
Precision is:  0.5789473684210527
Recall is:  0.6470588235294118
F1 score is:  0.6111111111111113
---------------------
---------------------
Int performance
Sensitivity is:  0.6363636363636364
Specificity is:  0.9745762711864406
Accuracy is:  0.9214285714285714
Error rate is:  0.07857142857142857
Precision is:  0.8235294117647058
Recall is:  0.6363636363636364
F1 score is:  0.717948717948718
---------------------


In [62]:
print_edu_fp(cs)


The average prediction score for false positive is:  0.7242391
---------------------
His doctorate research focused on human gait measurement and analysis using Ultra Wideband sensor systems.
---------------------
Prior to joining NUS he was employed at USP (The University of the South Pacific) in Suva, Fiji, where he lectured in the CS department there, specializing in Operating Systems and Networking.
---------------------
Subsequently, he was a postdoctoral associate at Princeton University and Rutgers University.
---------------------
From 1996 to 1998, he was Research Associate at the Sloan School of Management of the Massachusetts Institute of Technology (MIT) (United States of America).
---------------------

Trevor E. Carlson is an Assistant Professor of Computer Science at the School of Computing at the National University of Singapore (NUS).
---------------------
After that, he spent two years at Charles University, Prague, Czech Republic as a post-doctoral fellow hosted by P

In [63]:
print_awd_fp(cs)


The average prediction score for false positive is:  0.62540483
---------------------
I have given Keynote talks in IEEE/ACM International Conferences.


---------------------
In 1994, he was appointed site Leader of the Database Platform project and Principal Investigator and Work-package Manager for the European IDEA ESPRIT project on Intelligent Databases.
---------------------
He has been a regular invited speaker at universities and companies to share his work.
---------------------
He has also been a speaker at the NeurIPS'22, KDD'21, AAAI'21 and ICML'20 workshops on "Graph Representation Learning", and the ICLR'20 workshop on "Deep Neural Models and Differential Equations".
---------------------
His research has been published in ACM SIGMOD, VLDB, and IEEE ICDE Conferences.
---------------------
He has graduated 21 PhD students, served as the Associate Editor of IEEE Transaction on Mobile Computing, published more than 100 conference/journal papers and has been awarded 7 US pate

In [64]:
print_int_fp(cs)

The average prediction score for false positive is:  0.6597081
---------------------
She has many years of experience teaching Artificial Intelligence and Data Science courses such as machine learning, deep learning, and data mining, etc.


---------------------
Dr. Carlson??s research has been published at leading journals and conferences in computer architecture and simulation such as the International Symposium on Computer Architecture, the International Symposium on Microarchitecture, the International Symposium on High Performance Computer Architecture and the International Symposium on Performance Analysis of Systems and Software.


---------------------
He is a member of the Communication and Internet Research Lab, and his research areas cover sensor network protocols, mobile sensing and programmable network.
